In [1]:
%matplotlib inline
import cv2
import matplotlib.pyplot as plt
import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [2]:
import mxnet as mx
import numpy as np

from sklearn import svm
from sklearn import metrics
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

import os
import sys
import math
import time
import pickle

proj_dir = os.getcwd()
label_dir = os.path.join(proj_dir, 'emotiondetection/features_labels_lld/labels')
lld_dir = os.path.join(proj_dir, 'emotiondetection/features_labels_lld/lld')

pickle_train_y_list = 'pickle_train_y_list'
pickle_train_x_list = 'pickle_train_x_list'

pickle_test_y_list = 'pickle_test_y_list'
pickle_test_x_list = 'pickle_test_x_list'

In [3]:
train_y_list = pickle.load(open(pickle_train_y_list))
train_x_list = pickle.load(open(pickle_train_x_list))

test_y_list = pickle.load(open(pickle_test_y_list))
test_x_list = pickle.load(open(pickle_test_x_list))

In [4]:
def calc_score(test_y_list, predict_y_list):
    print "Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):"
    print 'macro   : ', metrics.precision_recall_fscore_support(test_y_list, predict_y_list, average='macro')
    print 'weighted: ', metrics.precision_recall_fscore_support(test_y_list, predict_y_list, average='weighted')

In [5]:
# z-score normalization
# scaler = preprocessing.StandardScaler().fit(train_x_list)
# train_x_list = scaler.transform(train_x_list)
# test_x_list = scaler.transform(test_x_list)

In [6]:
# PCA whiten
# pca = PCA(n_components='mle', svd_solver='full', whiten=True)
# x_pca = pca.fit(train_x_list)
# train_x_list = pca.transform(train_x_list)
# test_x_list = pca.transform(test_x_list)

In [7]:
# sklearn l1/l2 normalization
# x_normalizer = preprocessing.Normalizer(norm='l2').fit(train_x_list)
# train_x_list = x_normalizer.transform(train_x_list)
# test_x_list = x_normalizer.transform(test_x_list)

In [8]:
# # SMOTE up-sampling training data
# sm = SMOTE(kind='regular')
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)

# # split training data to 5 different classes
# train_x_lists = [[] for i in range(5)]
# for i in xrange(len(train_y_list)):
#     train_x_lists[int(train_y_list[i])].append(train_x_list[i])
# for i in xrange(5):
#     print 'number of class', i, ':', len(train_x_lists[i])

In [9]:
print train_y_list.shape, train_x_list.shape
print test_y_list.shape, test_x_list.shape

(9959,) (9959, 384)
(8257,) (8257, 384)


In [10]:
print train_x_list[0][:10]
print test_x_list[0][:10]

[  5.66733100e-03   3.11773100e-04   5.35555800e-03   5.30000000e+01
   9.60000000e+01   1.30774500e-03   4.18643100e-06   1.09633100e-03
   1.80239600e-06   1.34818000e-03]
[  1.23016500e-02   9.11361400e-05   1.22105100e-02   5.20000000e+01
   2.40000000e+01   1.31725500e-03   3.23349300e-07   1.29866300e-03
   6.89166500e-06   2.62522000e-03]


In [11]:
X = train_x_list
y = train_y_list

In [12]:
from hmmlearn.hmm import GaussianHMM
from hmmlearn.hmm import GMMHMM
from hmmlearn.hmm import MultinomialHMM
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [13]:
# split training data to 5 different classes
train_x_lists = [[] for i in range(5)]
for i in xrange(len(train_y_list)):
    train_x_lists[int(train_y_list[i])].append(train_x_list[i])
for i in xrange(5):
    print 'number of class', i, ':', len(train_x_lists[i])

number of class 0 : 881
number of class 1 : 2093
number of class 2 : 5590
number of class 3 : 674
number of class 4 : 721


In [14]:
n_components_list = [1, 2, 3, 4, 5]

for n2 in n_components_list:
    print '(GaussianHMM) ', 'n_components:', n2
    
    # train models and get scores
    models = []
    scores = []
    for i in range(5):
        X = np.array(train_x_lists[i])
        model = GaussianHMM(n_components=n2).fit(X)
        models.append(model)

        score = []
        for XX in test_x_list:
            score.append(model.score(np.array(XX)))
        scores.append(score)

    predict_y_list = np.argmax(np.array(scores).T, axis = 1)
    calc_score(test_y_list, predict_y_list.tolist())
    
    print '-----------------------'

(GaussianHMM)  n_components: 1
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.28728980247932873, 0.29514161618545598, 0.15357664546635824, None)
weighted:  (0.56070321596238637, 0.17778854305437811, 0.18989765836749217, None)
-----------------------
(GaussianHMM)  n_components: 2


/home/slhome/cc001/anaconda3/envs/python2/lib/python2.7/site-packages/hmmlearn/base.py:459: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),
/home/slhome/cc001/anaconda3/envs/python2/lib/python2.7/site-packages/hmmlearn/base.py:468: RuntimeWarning: divide by zero encountered in log
  np.log(self.startprob_),


Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.23124945373467454, 0.2807551659721319, 0.17641278784781542, None)
weighted:  (0.47811034446607742, 0.21521133583626015, 0.20276338535626062, None)
-----------------------
(GaussianHMM)  n_components: 3
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.22336560929426058, 0.27522527597090168, 0.16649205354690338, None)
weighted:  (0.46394158130893021, 0.20624924306648904, 0.17062070148257513, None)
-----------------------
(GaussianHMM)  n_components: 4
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.2139925820903153, 0.24337127084175547, 0.14904008502583613, None)
weighted:  (0.46635329806800363, 0.17633523071333415, 0.15568076556927068, None)
-----------------------
(GaussianHMM)  n_components: 5
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.22983865651732063, 0.24819941423725661, 0.20030795726337933, None)
weighted:  (0.4759823028414561, 0.3097977473658714

In [16]:
n_mix_list = [1, 2, 3, 4, 5, 6, 7, 8]
n_components_list = [1, 2, 3]

for n1 in n_mix_list:
    for n2 in n_components_list:
        print '(GMMHMM) ', 'n_mix:', n1, ';',  'n_components:', n2
        
        # train models and get scores
        models = []
        scores = []
        for i in range(5):
            X = np.array(train_x_lists[i])
            model = GMMHMM(n_mix=n1, n_components=n2).fit(X)
            models.append(model)

            score = []
            for XX in test_x_list:
                score.append(model.score(np.array(XX)))
            scores.append(score)

        predict_y_list = np.argmax(np.array(scores).T, axis = 1)
        calc_score(test_y_list, predict_y_list.tolist())
        
        print '-----------------------'

(GMMHMM)  n_mix: 1 ; n_components: 1
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.28728980247932873, 0.29514161618545598, 0.15357664546635824, None)
weighted:  (0.56070321596238637, 0.17778854305437811, 0.18989765836749217, None)
-----------------------
(GMMHMM)  n_mix: 1 ; n_components: 2
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.28656992711534712, 0.29793731825852249, 0.16713617613912013, None)
weighted:  (0.5569126877925743, 0.20116264987283516, 0.22406732210688879, None)
-----------------------
(GMMHMM)  n_mix: 1 ; n_components: 3
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.28862651446405807, 0.30158902756695516, 0.17770008782262661, None)
weighted:  (0.55768509838665115, 0.21884461668887004, 0.2485959143829633, None)
-----------------------
(GMMHMM)  n_mix: 2 ; n_components: 1


/home/slhome/cc001/anaconda3/envs/python2/lib/python2.7/site-packages/hmmlearn/hmm.py:593: RuntimeWarning: divide by zero encountered in log
  lgmm_posteriors = (np.log(g.predict_proba(X))


Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.27634101141811268, 0.31460990086594176, 0.26071791554771334, None)
weighted:  (0.5563068028633007, 0.42460942230834442, 0.45845600089253763, None)
-----------------------
(GMMHMM)  n_mix: 2 ; n_components: 2
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.27748533441380541, 0.31453976036678311, 0.26905579411583591, None)
weighted:  (0.55407251926277667, 0.45440232529974567, 0.48261956383637061, None)
-----------------------
(GMMHMM)  n_mix: 2 ; n_components: 3
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.2684947029092693, 0.27432566947490394, 0.24698637997724396, None)
weighted:  (0.55334007684077902, 0.38924548867627468, 0.44135661629610495, None)
-----------------------
(GMMHMM)  n_mix: 3 ; n_components: 1
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.27417918542938335, 0.31387640374488923, 0.26945496919214645, None)
weighted:  (0.55337153795565452,

In [17]:
# SMOTE up-sampling training data
sm = SMOTE(kind='regular')
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list, train_y_list)
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list_smote, train_y_list_smote)
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list_smote, train_y_list_smote)
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list_smote, train_y_list_smote)
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list_smote, train_y_list_smote)

# split training data to 5 different classes
train_x_lists = [[] for i in range(5)]
for i in xrange(len(train_y_list_smote)):
    train_x_lists[int(train_y_list_smote[i])].append(train_x_list_smote[i])
for i in xrange(5):
    print 'number of class', i, ':', len(train_x_lists[i])

No handlers could be found for logger "imblearn.base"


number of class 0 : 5590
number of class 1 : 5590
number of class 2 : 5590
number of class 3 : 5590
number of class 4 : 5590


In [18]:
n_components_list = [1, 2, 3, 4, 5]

for n2 in n_components_list:
    print '(GaussianHMM) ', 'n_components:', n2
    
    # train models and get scores
    models = []
    scores = []
    for i in range(5):
        X = np.array(train_x_lists[i])
        model = GaussianHMM(n_components=n2).fit(X)
        models.append(model)

        score = []
        for XX in test_x_list:
            score.append(model.score(np.array(XX)))
        scores.append(score)

    predict_y_list = np.argmax(np.array(scores).T, axis = 1)
    calc_score(test_y_list, predict_y_list.tolist())
    
    print '-----------------------'

(GaussianHMM)  n_components: 1
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.28294100176656489, 0.31557044496354125, 0.16807868418435262, None)
weighted:  (0.56220098176125477, 0.18432844858907593, 0.19531911519038486, None)
-----------------------
(GaussianHMM)  n_components: 2
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.22844463449203056, 0.29862826415004512, 0.17521774361387382, None)
weighted:  (0.46440762536614943, 0.1863873077388882, 0.18566820569451298, None)
-----------------------
(GaussianHMM)  n_components: 3
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.23296009400181011, 0.28639739230569955, 0.18459541589561093, None)
weighted:  (0.47851196292376807, 0.2273222720116265, 0.2060594527259163, None)
-----------------------
(GaussianHMM)  n_components: 4
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.22429321174130018, 0.25074332558660206, 0.16024935921574635, None)
weighted:  (0.473472

In [19]:
n_mix_list = [1, 2, 3, 4, 5, 6, 7, 8]
n_components_list = [1, 2, 3]

for n1 in n_mix_list:
    for n2 in n_components_list:
        print '(GMMHMM) ', 'n_mix:', n1, ';',  'n_components:', n2
        
        # train models and get scores
        models = []
        scores = []
        for i in range(5):
            X = np.array(train_x_lists[i])
            model = GMMHMM(n_mix=n1, n_components=n2).fit(X)
            models.append(model)

            score = []
            for XX in test_x_list:
                score.append(model.score(np.array(XX)))
            scores.append(score)

        predict_y_list = np.argmax(np.array(scores).T, axis = 1)
        calc_score(test_y_list, predict_y_list.tolist())
        
        print '-----------------------'

(GMMHMM)  n_mix: 1 ; n_components: 1
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.28294100176656489, 0.31557044496354125, 0.16807868418435262, None)
weighted:  (0.56220098176125477, 0.18432844858907593, 0.19531911519038486, None)
-----------------------
(GMMHMM)  n_mix: 1 ; n_components: 2
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.28282766627291528, 0.31552444263127088, 0.1691379950351852, None)
weighted:  (0.56331556303866492, 0.1854184328448589, 0.19658480367625403, None)
-----------------------
(GMMHMM)  n_mix: 1 ; n_components: 3
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.28204536707510763, 0.31604621767328794, 0.17007216221395985, None)
weighted:  (0.56120213508542105, 0.18650841710064189, 0.19786537910095009, None)
-----------------------
(GMMHMM)  n_mix: 2 ; n_components: 1
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.26032677749414174, 0.31568093090505944, 0.20628807302343391, N

In [20]:
print predict_y_list[:100]

[2 1 2 2 2 0 2 1 1 1 4 2 3 2 1 1 1 1 1 4 1 2 0 0 2 0 0 2 1 0 2 4 0 4 1 2 1
 1 0 0 1 0 0 0 1 1 1 1 1 0 0 0 0 1 1 1 4 2 0 0 0 2 0 4 0 2 1 0 2 2 1 0 1 0
 1 0 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 0 2 1 1 0 0 1 1 0]


In [21]:
print test_y_list[:100]

[2 2 2 2 2 4 2 2 2 2 2 2 2 2 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1
 1 2 2 2 2 2 2 2 1 1 2 1 1 2 2 2 0 1 2 2 2 2 2 2 2 2 2 2 1 1 2 2 2 2 2 2 2
 1 1 1 2 1 1 1 1 1 2 2 1 1 1 2 1 1 2 2 2 2 2 1 1 1 2]
